In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.api import ExponentialSmoothing
from itertools import product
from typing import Literal, Union, Callable
import warnings
from functools import partial

import sys
sys.path.insert(1, '../src')

from rolling_window import RollingWindow, basic_rollapply
from ts_funs import exp_smooth_wrapper, metric_eval_wrapper, calc_score 

SEED = 1234

In [2]:
df = pd.read_parquet("../data/cases_daily.parquet")
df.head()

,date,fips,cases,deaths
0,2020-01-21,1001,0,0
1,2020-01-22,1001,0,0
2,2020-01-23,1001,0,0
3,2020-01-24,1001,0,0
4,2020-01-25,1001,0,0


## Country-level analysis

In [3]:
x = df.groupby("date")["cases"].sum()
x[:5]

date
2020-01-21    1
2020-01-22    0
2020-01-23    0
2020-01-24    1
2020-01-25    1
Name: cases, dtype: int64

In [4]:
import plotly.express as px

px.line(x)


### Tuning

In [5]:
def expand_grid(d: dict):
   return pd.DataFrame([row for row in product(*d.values())], 
                       columns=d.keys())



In [6]:
params_grid = expand_grid({
    "trend": ["add", "mul"],
    "damped_trend": [True, False],
    "seasonal": ["add", "mul"],
    "initialization_method": ["estimated", "heuristic"],
    "use_boxcox": [True, False]
})
params_grid.head()

,trend,damped_trend,seasonal,initialization_method,use_boxcox
0,add,True,add,estimated,True
1,add,True,add,estimated,False
2,add,True,add,heuristic,True
3,add,True,add,heuristic,False
4,add,True,mul,estimated,True


In [7]:
FRAC = 0.15
HORIZON_FORECAST = 30

test_size = round(FRAC * len(x))
val_size = round(FRAC * (1 - FRAC) * len(x))

window_size = len(x) - val_size - test_size

x_train1 = x[:(len(x) - test_size)]
x_train2 = x[val_size:]

len(x_train1), len(x_train2)

(734, 754)

In [8]:
rw_tuning = RollingWindow(
    x_train1,
    window_size,
    HORIZON_FORECAST
)

rw_eval = RollingWindow(
    x_train2,
    window_size,
    HORIZON_FORECAST
)

In [9]:
fn = partial(calc_score, model_wrapper=exp_smooth_wrapper, rolling_window=rw_tuning, forecast_horizon=HORIZON_FORECAST)

In [10]:
metrics = np.array([], dtype=np.double)
for p in params_grid.to_dict("records"):
    metrics = np.append(metrics, [fn(p)])


In [17]:
params_grid["MAPE"] = metrics
df_best_params = params_grid.sort_values("MAPE", ascending=True).head(5)
df_best_params

,trend,damped_trend,seasonal,initialization_method,use_boxcox,MAPE
0,add,True,add,estimated,True,0.346966
2,add,True,add,heuristic,True,0.346966
13,add,False,mul,estimated,False,0.361512
15,add,False,mul,heuristic,False,0.361512
5,add,True,mul,estimated,False,0.369168


In [19]:
df_best_params.to_csv("../data/best_exp_smoothing_params.csv", index=False)

### Evaluation

In [12]:
best_params = df_best_params.drop(columns=["MAPE"]).head(1).to_dict("records")[0]
best_params

{'trend': 'add',
 'damped_trend': True,
 'seasonal': 'add',
 'initialization_method': 'estimated',
 'use_boxcox': True}

In [13]:
calc_score(
    best_params,
    model_wrapper=exp_smooth_wrapper,
    rolling_window=rw_eval,
    forecast_horizon=HORIZON_FORECAST
)

0.5737392351239435

In [14]:
x1 = x[val_size:-test_size]
x2 = x[-window_size:]

fcst1 = exp_smooth_wrapper(x1, HORIZON_FORECAST, **best_params)
fcst2 = exp_smooth_wrapper(x2, HORIZON_FORECAST, **best_params)

In [15]:
fcst1.index.dtype

dtype('<M8[ns]')

In [16]:
# plot_df = x \
#     .to_frame() \
#     .join(fcst1.rename("Forecast 1")) \
#     .join(fcst2.rename("Forecast 2"))

x.index = pd.to_datetime(x.index)

plot_df = x \
    .to_frame() \
    .join(fcst1.rename("Forecast 1"), on="date", how="outer") \
    .join(fcst2.rename("Forecast 2"), on="date", how="outer") \
    .reset_index() \
    .drop(columns=["index"])

plot_df.head()

,date,cases,Forecast 1,Forecast 2
0,2020-01-21,1.0,NaN,NaN
1,2020-01-22,0.0,NaN,NaN
2,2020-01-23,0.0,NaN,NaN
3,2020-01-24,1.0,NaN,NaN
4,2020-01-25,1.0,NaN,NaN


In [34]:
plot_df.to_csv("../data/country_level_cases_with_forecast.csv", index=False)

In [88]:
px.line(
    plot_df,
    x="date",
    y = ["cases", "Forecast 1", "Forecast 2"]
)

## State-level analysis

In [27]:
df_fips_states = pd.read_csv("../data/fips_states.csv", dtype={"fips_state": str})
df_fips_states.state_name = df_fips_states.state_name.str.title()
df_fips_states.head()

,fips_state,state_name
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


In [28]:
df_states = df.copy()
df_states["fips_state"] = [f"{fips:05d}"[:2] for fips in df_states.fips]

df_states = pd.merge(
    df_states,
    df_fips_states,
    how="left",
    on="fips_state"
).drop(columns=["fips_state"])

df_states.head()


,date,fips,cases,deaths,state_name
0,2020-01-21,1001,0,0,Alabama
1,2020-01-22,1001,0,0,Alabama
2,2020-01-23,1001,0,0,Alabama
3,2020-01-24,1001,0,0,Alabama
4,2020-01-25,1001,0,0,Alabama


In [29]:
state_dict = {}

for state, df in df_states.groupby("state_name"):
    state_dict[state] = df.groupby("date")["cases"].sum()


In [33]:
results_list = []
for state_name, df_state in state_dict.items():
    train = df_state[val_size:]
    rw = RollingWindow(train, window_size, HORIZON_FORECAST)

    score = calc_score(
        best_params,
        exp_smooth_wrapper,
        rw, 
        HORIZON_FORECAST
    )

    results_list.append({"state": state_name, "MAPE": score})

results_df = pd.DataFrame.from_records(results_list)
results_df.head()

,state,MAPE
0,Alabama,6.165946e+17
1,Alaska,2.025908e+17
2,Arizona,1.352917e+18
3,Arkansas,1.103856e+17
4,California,4.989235e+17
5,Colorado,7.583249e+16
6,Connecticut,1.364066e+16
7,Delaware,4.797125e+16
8,District Of Columbia,1.290447e+17
9,Florida,2.848113e+18


In [35]:
results_df.MAPE[0]

6.165945755946707e+17